In [1]:
# Instalar librerías hana
import hana_ml
import hana_ml.dataframe as dataframe
import os
from dotenv import load_dotenv
import sys

# Instalar librerías de visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Instalar librerías de machine learning
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
import datetime

print("Librerías importadas correctamente")


Librerías importadas correctamente


In [2]:
# CONFIGURATIONS:

print("HANNA version:", hana_ml.__version__)
load_dotenv('../.env')

HANNA version: 2.24.25051600


True

In [3]:
conn = dataframe.ConnectionContext(address=os.getenv('HANA_HOST'),
                                    port=443,
                                    user=os.getenv('HANA_USER'),
                                    password=os.getenv('HANA_PASSWORD')
                                    )

conn.connection.isconnected()

True

In [4]:
# Verificar la conexión
print("Conexión establecida:", conn.connection.isconnected())
print("Esquema actual:", conn.get_current_schema())

Conexión establecida: True
Esquema actual: DBADMIN


In [5]:
'''
-- 1.1 Tabla de Roles
CREATE TABLE Rol2 (
    Rol_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Nombre NVARCHAR(100) NOT NULL
);

-- 1.2 Tabla de Locations
CREATE TABLE Location2 (
    Location_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Nombre NVARCHAR(100),
    Tipo NVARCHAR(50),
    PosicionX INTEGER,
    PosicionY INTEGER,
    FechaCreado DATE
);

-- 1.3 Tabla de Usuarios
CREATE TABLE Usuario2 (
    Usuario_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Correo NVARCHAR(255),
    Username NVARCHAR(100),
    Nombre NVARCHAR(100) NOT NULL,
    Rol_ID INTEGER,
    Clave NVARCHAR(255) NOT NULL,
    Location_ID INTEGER,
    FechaEmpiezo DATE,
    RFC NVARCHAR(13),
    FOREIGN KEY (Rol_ID) REFERENCES Rol2(Rol_ID),
    FOREIGN KEY (Location_ID) REFERENCES Location2(Location_ID)
);

-- 1.4 Tabla de Artículos
CREATE TABLE Articulo2 (
    Articulo_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Nombre NVARCHAR(100) NOT NULL,
    Categoria NVARCHAR(100),
    PrecioProveedor DECIMAL(10,2),
    PrecioVenta DECIMAL(10,2),
    Temporada NVARCHAR(50)
);

-- 1.5 Tabla de Inventario
CREATE TABLE Inventario2 (
    Inventario_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Articulo_ID INTEGER,
    Location_ID INTEGER,
    StockActual INTEGER,
    Importacion INTEGER,
    Exportacion INTEGER,
    StockMinimo INTEGER,
    StockRecomendado INTEGER,
    FechaUltimaImportacion DATE,
    FechaUltimaExportacion DATE,
    MargenGanancia DECIMAL(5,2),
    TiempoReposicion DECIMAL(5,2),
    StockSeguridad INTEGER,
    DemandaPromedio DECIMAL(5,2),
    FOREIGN KEY (Articulo_ID) REFERENCES Articulo2(Articulo_ID),
    FOREIGN KEY (Location_ID) REFERENCES Location2(Location_ID)
);

-- 1.6 Tabla de Fabricación
CREATE TABLE Fabricacion2 (
    Fabricacion_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Location_ID INTEGER,
    Articulo_ID INTEGER,
    FOREIGN KEY (Location_ID) REFERENCES Location2(Location_ID),
    FOREIGN KEY (Articulo_ID) REFERENCES Articulo2(Articulo_ID)
);

-- 1.7 Tabla de Métodos de Pago
CREATE TABLE MetodoPago2 (
    MetodoPago_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Nombre NVARCHAR(100) NOT NULL
);

-- 1.8 Tabla de Órdenes
CREATE TABLE Ordenes2 (
    Orden_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Creado_por_ID INTEGER,
    Modificado_por_ID INTEGER,
    TipoOrden NVARCHAR(50),
    Organizacion NVARCHAR(100),
    FechaCreacion DATE,
    FechaAceptacion DATE,
    FechaLimitePago DATE,
    FechaEstimadaEntrega DATE,
    FechaEntrega DATE,
    EntregaATiempo BOOLEAN,
    Estado NVARCHAR(50),
    Total DECIMAL(10,2),
    MetodoPago_ID INTEGER,
    DescuentoAplicado DECIMAL(5,2),
    TiempoReposicion DECIMAL(5,2),
    TiempoEntrega DECIMAL(5,2),
    FOREIGN KEY (Creado_por_ID) REFERENCES Usuario2(Usuario_ID),
    FOREIGN KEY (Modificado_por_ID) REFERENCES Usuario2(Usuario_ID),
    FOREIGN KEY (MetodoPago_ID) REFERENCES MetodoPago2(MetodoPago_ID)
);

-- 1.9 Tabla de OrdenesProductos
CREATE TABLE OrdenesProductos2 (
    OrdenesProductos_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Orden_ID INTEGER,
    Inventario_ID INTEGER,
    Cantidad INTEGER,
    PrecioUnitario DECIMAL(10,2),
    FOREIGN KEY (Orden_ID) REFERENCES Ordenes2(Orden_ID),
    FOREIGN KEY (Inventario_ID) REFERENCES Inventario2(Inventario_ID)
);

-- 1.10 Tabla de Comentarios de Órdenes
CREATE TABLE ComentariosOrdenes2 (
    Comentario_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Orden_ID INTEGER,
    Creado_por_ID INTEGER,
    Comentario NCLOB,
    FechaCreado TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (Orden_ID) REFERENCES Ordenes2(Orden_ID),
    FOREIGN KEY (Creado_por_ID) REFERENCES Usuario2(Usuario_ID)
);

-- 1.11 Tabla de Pagos por Orden
CREATE TABLE PagoOrden2 (
    Pago_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Orden_ID INTEGER,
    MetodoPago_ID INTEGER,
    EstadoPago NVARCHAR(50),
    ComentarioPago NCLOB,
    FechaPago DATE,
    DescuentoAplicado DECIMAL(5,2),
    FOREIGN KEY (Orden_ID) REFERENCES Ordenes2(Orden_ID),
    FOREIGN KEY (MetodoPago_ID) REFERENCES MetodoPago2(MetodoPago_ID)
);

-- 1.12 Tabla de Historial de Productos
CREATE TABLE HistorialProductos2 (
    Historial_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Inventario_ID INTEGER,
    Location_ID INTEGER,
    Anio INTEGER,
    Mes INTEGER,
    Importacion INTEGER,
    Exportacion INTEGER,
    StockStart INTEGER,
    StockEnd INTEGER,
    FOREIGN KEY (Inventario_ID) REFERENCES Inventario2(Inventario_ID),
    FOREIGN KEY (Location_ID) REFERENCES Location2(Location_ID)
);

-- 1.13 Tabla de Alertas
CREATE TABLE Alertas2 (
    Alerta_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Descripcion NCLOB,
    FechaCreacion DATE,
    OrdenesProductos_ID INTEGER,
    Location_ID INTEGER,
    FOREIGN KEY (OrdenesProductos_ID) REFERENCES OrdenesProductos2(OrdenesProductos_ID),
    FOREIGN KEY (Location_ID) REFERENCES Location2(Location_ID)
);

-- 1.14 Tabla Bitácora Maestra
CREATE TABLE Bitacora_Maestra2 (
    Bitacora_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    Nombre NVARCHAR(100)
);

-- 1.15 Tabla Bitácora General
CREATE TABLE Bitacora_General2 (
    General_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    MaestraID INTEGER,
    CampoID INTEGER,
    Nombre NVARCHAR(100),
    Descripcion NCLOB,
    FOREIGN KEY (MaestraID) REFERENCES Bitacora_Maestra2(Bitacora_ID)
);

'''

'\n-- 1.1 Tabla de Roles\nCREATE TABLE Rol2 (\n    Rol_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,\n    Nombre NVARCHAR(100) NOT NULL\n);\n\n-- 1.2 Tabla de Locations\nCREATE TABLE Location2 (\n    Location_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,\n    Nombre NVARCHAR(100),\n    Tipo NVARCHAR(50),\n    PosicionX INTEGER,\n    PosicionY INTEGER,\n    FechaCreado DATE\n);\n\n-- 1.3 Tabla de Usuarios\nCREATE TABLE Usuario2 (\n    Usuario_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,\n    Correo NVARCHAR(255),\n    Username NVARCHAR(100),\n    Nombre NVARCHAR(100) NOT NULL,\n    Rol_ID INTEGER,\n    Clave NVARCHAR(255) NOT NULL,\n    Location_ID INTEGER,\n    FechaEmpiezo DATE,\n    RFC NVARCHAR(13),\n    FOREIGN KEY (Rol_ID) REFERENCES Rol2(Rol_ID),\n    FOREIGN KEY (Location_ID) REFERENCES Location2(Location_ID)\n);\n\n-- 1.4 Tabla de Artículos\nCREATE TABLE Articulo2 (\n    Articulo_ID INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,\n    Nom

In [6]:
# Verificar la existencia y estructura de las tablas utilizadas
# Obtener lista de tablas en el esquema DBADMIN
def get_table_info(conn, table_name, schema='DBADMIN', check_case_variations=True):
    try:
        # Verificar si la tabla existe exactamente como se especificó
        table_exists_query = f"SELECT COUNT(*) as TABLE_EXISTS FROM SYS.TABLES WHERE SCHEMA_NAME = '{schema}' AND TABLE_NAME = '{table_name}'"
        table_exists_result = conn.sql(table_exists_query).collect()
        table_exists = table_exists_result['TABLE_EXISTS'][0] > 0
        
        if table_exists:
            # Obtener la estructura de la tabla
            columns_query = f"SELECT COLUMN_NAME, DATA_TYPE_NAME FROM SYS.TABLE_COLUMNS WHERE SCHEMA_NAME = '{schema}' AND TABLE_NAME = '{table_name}' ORDER BY POSITION"
            columns = conn.sql(columns_query).collect()
            
            return {
                'exists': True,
                'actual_name': table_name,
                'columns': columns
            }
        elif check_case_variations:
            # Si la tabla no existe con el nombre exacto, buscar variaciones de mayúsculas/minúsculas
            case_insensitive_query = f"SELECT TABLE_NAME FROM SYS.TABLES WHERE SCHEMA_NAME = '{schema}' AND UPPER(TABLE_NAME) = UPPER('{table_name}')"
            case_result = conn.sql(case_insensitive_query).collect()
            
            if not case_result.empty:
                actual_name = case_result['TABLE_NAME'][0]
                print(f"\n🔄 Tabla encontrada con variación de mayúsculas/minúsculas: '{table_name}' → '{actual_name}'")
                
                # Obtener la estructura de la tabla
                columns_query = f"SELECT COLUMN_NAME, DATA_TYPE_NAME FROM SYS.TABLE_COLUMNS WHERE SCHEMA_NAME = '{schema}' AND TABLE_NAME = '{actual_name}' ORDER BY POSITION"
                columns = conn.sql(columns_query).collect()
                
                return {
                    'exists': True,
                    'actual_name': actual_name,
                    'columns': columns,
                    'case_mismatch': True
                }
            
            # También buscar nombres similares que podrían ser lo que el usuario busca
            similar_query = f"""SELECT TABLE_NAME 
                            FROM SYS.TABLES 
                            WHERE SCHEMA_NAME = '{schema}' 
                            AND (
                                TABLE_NAME LIKE '%{table_name}%' OR
                                '{table_name}' LIKE '%' || TABLE_NAME || '%'
                            )
                            ORDER BY LENGTH(TABLE_NAME)"""
            similar_result = conn.sql(similar_query).collect()
            
            if not similar_result.empty:
                similar_tables = similar_result['TABLE_NAME'].tolist()
                print(f"\n🔍 No se encontró '{table_name}', pero hay tablas con nombres similares: {', '.join(similar_tables[:5])}")
                if len(similar_tables) > 5:
                    print(f"   ...y {len(similar_tables) - 5} más")
        
        return {'exists': False, 'columns': []}
    except Exception as e:
        print(f"Error al verificar tabla {table_name}: {e}")
        return {'exists': False, 'error': str(e), 'columns': []}

# Tablas a verificar - Solo se usarán tablas con sufijo 2
tablas = [
    'INVENTARIO2', 
    'HISTORIALPRODUCTOS2', 
    'ARTICULO2', 
    'ORDENESPRODUCTOS2', 
    'ORDENES2'
]

resultados = {}
for tabla in tablas:
    info = get_table_info(conn, tabla)
    resultados[tabla] = info
    
    if info['exists']:
        nombre_real = info.get('actual_name', tabla)
        if info.get('case_mismatch', False):
            print(f"✅ Tabla {tabla} existe como '{nombre_real}' (considere usar este nombre exacto)")
        else:
            print(f"✅ Tabla {tabla} existe")
        print(f"   Columnas: {len(info['columns'])}")
    else:
        print(f"❌ Tabla {tabla} no existe")
        
# Mostrar detalles de las columnas para las tablas que existen
print("\nDetalle de columnas por tabla:")
for tabla, info in resultados.items():
    if info['exists']:
        print(f"\n=== {tabla} ===")
        display(info['columns'])

# Verificar si las columnas mencionadas en el código existen en las tablas
tablas_columnas = {
    'INVENTARIO2': [
        'Inventario_ID', 'Articulo_ID', 'Location_ID', 'StockActual', 'StockMinimo', 
        'StockRecomendado', 'MargenGanancia', 'TiempoReposicion', 'StockSeguridad', 'DemandaPromedio'
    ],
    'HISTORIALPRODUCTOS2': ['Inventario_ID', 'Location_ID', 'Anio', 'Mes', 'Importacion', 'Exportacion', 'StockEnd', 'StockStart'],
    'ARTICULO2': ['Articulo_ID', 'Nombre', 'Categoria', 'PrecioProveedor', 'PrecioVenta', 'Temporada'],
    'ORDENESPRODUCTOS2': ['OrdenesProductos_ID', 'Inventario_ID', 'Orden_ID', 'Cantidad', 'PrecioUnitario'],
    'ORDENES2': ['Orden_ID', 'FechaCreacion', 'FechaEntrega', 'TipoOrden']
}

# Verificar que las columnas necesarias existan
print("\nVerificación de columnas necesarias:")
for tabla, columnas in tablas_columnas.items():
    if tabla in resultados and resultados[tabla]['exists']:
        columns_df = resultados[tabla]['columns']
        columnas_existentes = columns_df['COLUMN_NAME'].tolist()
        
        print(f"\n=== {tabla} ===")
        for col in columnas:
            if col in columnas_existentes:
                print(f"✅ {col}")
            else:
                # Buscar variaciones de mayúsculas/minúsculas para la columna
                variaciones = [c for c in columnas_existentes if c.upper() == col.upper()]
                if variaciones:
                    print(f"⚠️ {col} - Existe como '{variaciones[0]}'")
                else:
                    print(f"❌ {col} - No existe")
    else:
        print(f"\n=== {tabla} ===")
        print("❌ Tabla no existe")

✅ Tabla INVENTARIO2 existe
   Columnas: 14
✅ Tabla HISTORIALPRODUCTOS2 existe
   Columnas: 9
✅ Tabla HISTORIALPRODUCTOS2 existe
   Columnas: 9
✅ Tabla ARTICULO2 existe
   Columnas: 6
✅ Tabla ARTICULO2 existe
   Columnas: 6
✅ Tabla ORDENESPRODUCTOS2 existe
   Columnas: 5
✅ Tabla ORDENESPRODUCTOS2 existe
   Columnas: 5
✅ Tabla ORDENES2 existe
   Columnas: 17

Detalle de columnas por tabla:

=== INVENTARIO2 ===
✅ Tabla ORDENES2 existe
   Columnas: 17

Detalle de columnas por tabla:

=== INVENTARIO2 ===


,COLUMN_NAME,DATA_TYPE_NAME
0,INVENTARIO_ID,INTEGER
1,ARTICULO_ID,INTEGER
2,LOCATION_ID,INTEGER
3,STOCKACTUAL,INTEGER
4,IMPORTACION,INTEGER
5,EXPORTACION,INTEGER
6,STOCKMINIMO,INTEGER
7,STOCKRECOMENDADO,INTEGER
8,FECHAULTIMAIMPORTACION,DATE
9,FECHAULTIMAEXPORTACION,DATE



=== HISTORIALPRODUCTOS2 ===


,COLUMN_NAME,DATA_TYPE_NAME
0,HISTORIAL_ID,INTEGER
1,INVENTARIO_ID,INTEGER
2,LOCATION_ID,INTEGER
3,ANIO,INTEGER
4,MES,INTEGER
5,IMPORTACION,INTEGER
6,EXPORTACION,INTEGER
7,STOCKSTART,INTEGER
8,STOCKEND,INTEGER



=== ARTICULO2 ===


...


=== ORDENESPRODUCTOS2 ===


,COLUMN_NAME,DATA_TYPE_NAME
0,ORDENESPRODUCTOS_ID,INTEGER
1,ORDEN_ID,INTEGER
2,INVENTARIO_ID,INTEGER
3,CANTIDAD,INTEGER
4,PRECIOUNITARIO,DECIMAL



=== ORDENES2 ===


,COLUMN_NAME,DATA_TYPE_NAME
0,ORDEN_ID,INTEGER
1,CREADO_POR_ID,INTEGER
2,MODIFICADO_POR_ID,INTEGER
3,TIPOORDEN,NVARCHAR
4,ORGANIZACION,NVARCHAR
5,FECHACREACION,DATE
6,FECHAACEPTACION,DATE
7,FECHALIMITEPAGO,DATE
8,FECHAESTIMADAENTREGA,DATE
9,FECHAENTREGA,DATE



Verificación de columnas necesarias:

=== INVENTARIO2 ===
⚠️ Inventario_ID - Existe como 'INVENTARIO_ID'
⚠️ Articulo_ID - Existe como 'ARTICULO_ID'
⚠️ Location_ID - Existe como 'LOCATION_ID'
⚠️ StockActual - Existe como 'STOCKACTUAL'
⚠️ StockMinimo - Existe como 'STOCKMINIMO'
⚠️ StockRecomendado - Existe como 'STOCKRECOMENDADO'
⚠️ MargenGanancia - Existe como 'MARGENGANANCIA'
⚠️ TiempoReposicion - Existe como 'TIEMPOREPOSICION'
⚠️ StockSeguridad - Existe como 'STOCKSEGURIDAD'
⚠️ DemandaPromedio - Existe como 'DEMANDAPROMEDIO'

=== HISTORIALPRODUCTOS2 ===
⚠️ Inventario_ID - Existe como 'INVENTARIO_ID'
⚠️ Location_ID - Existe como 'LOCATION_ID'
⚠️ Anio - Existe como 'ANIO'
⚠️ Mes - Existe como 'MES'
⚠️ Importacion - Existe como 'IMPORTACION'
⚠️ Exportacion - Existe como 'EXPORTACION'
⚠️ StockEnd - Existe como 'STOCKEND'
⚠️ StockStart - Existe como 'STOCKSTART'

=== ARTICULO2 ===
⚠️ Articulo_ID - Existe como 'ARTICULO_ID'
⚠️ Nombre - Existe como 'NOMBRE'
⚠️ Categoria - Existe como 'CATEG

In [7]:
# API de DataFrame de HANA ML
# Cargar las tablas como DataFrames de HANA ML usando los nombres determinados en la celda anterior

# Obtener los nombres reales de las tablas (con mayúsculas/minúsculas correctas)
tabla_inventario = resultados['INVENTARIO2']['actual_name'] if resultados['INVENTARIO2']['exists'] else 'INVENTARIO2'
tabla_historial = resultados['HISTORIALPRODUCTOS2']['actual_name'] if resultados['HISTORIALPRODUCTOS2']['exists'] else 'HISTORIALPRODUCTOS2'
tabla_articulo = resultados['ARTICULO2']['actual_name'] if resultados['ARTICULO2']['exists'] else 'ARTICULO2'
tabla_ordenes_productos = resultados['ORDENESPRODUCTOS2']['actual_name'] if resultados['ORDENESPRODUCTOS2']['exists'] else 'ORDENESPRODUCTOS2'
tabla_ordenes = resultados['ORDENES2']['actual_name'] if resultados['ORDENES2']['exists'] else 'ORDENES2'

print(f"Cargando tablas con los nombres reales encontrados:")
print(f"- Inventario: {tabla_inventario}")
print(f"- Historial: {tabla_historial}")
print(f"- Articulo: {tabla_articulo}")
print(f"- Ordenes Productos: {tabla_ordenes_productos}")
print(f"- Ordenes: {tabla_ordenes}")

# Verificar que todas las tablas existan antes de continuar
tablas_requeridas = {
    'INVENTARIO2': tabla_inventario,
    'HISTORIALPRODUCTOS2': tabla_historial,
    'ARTICULO2': tabla_articulo, 
    'ORDENESPRODUCTOS2': tabla_ordenes_productos,
    'ORDENES2': tabla_ordenes
}

# Cargar las tablas como DataFrames de HANA ML
inventario_df = conn.table(tabla_inventario, schema='DBADMIN')
historial_df = conn.table(tabla_historial, schema='DBADMIN')
articulo_df = conn.table(tabla_articulo, schema='DBADMIN')
ordenes_productos_df = conn.table(tabla_ordenes_productos, schema='DBADMIN')
ordenes_df = conn.table(tabla_ordenes, schema='DBADMIN')

Cargando tablas con los nombres reales encontrados:
- Inventario: INVENTARIO2
- Historial: HISTORIALPRODUCTOS2
- Articulo: ARTICULO2
- Ordenes Productos: ORDENESPRODUCTOS2
- Ordenes: ORDENES2


In [8]:
# Transformar los DataFrames de HANA ML a DataFrames de pandas para visualización
inventario_df = inventario_df.collect()
historial_df = historial_df.collect()
articulo_df = articulo_df.collect()
ordenes_productos_df = ordenes_productos_df.collect()
ordenes_df = ordenes_df.collect()

In [9]:
inventario_df.head(5)

,INVENTARIO_ID,ARTICULO_ID,LOCATION_ID,STOCKACTUAL,IMPORTACION,EXPORTACION,STOCKMINIMO,STOCKRECOMENDADO,FECHAULTIMAIMPORTACION,FECHAULTIMAEXPORTACION,MARGENGANANCIA,TIEMPOREPOSICION,STOCKSEGURIDAD,DEMANDAPROMEDIO
0,1,1,8,120,0,0,30,50,2025-04-01,2025-04-20,50,5,20,40
1,2,2,8,100,0,0,25,50,2025-04-01,2025-04-19,50,5,20,35
2,3,3,8,80,0,0,20,50,2025-04-01,2025-04-18,50,5,15,30
3,4,4,9,60,0,0,15,50,2025-04-03,2025-04-21,50,6,10,25
4,5,5,9,70,0,0,20,50,2025-04-03,2025-04-21,50,6,12,28


In [10]:
historial_df.head(5)

,HISTORIAL_ID,INVENTARIO_ID,LOCATION_ID,ANIO,MES,IMPORTACION,EXPORTACION,STOCKSTART,STOCKEND
0,1,1,8,2025,4,120,80,150,120
1,2,2,8,2025,4,100,65,130,100


In [11]:
articulo_df.head(5)

,ARTICULO_ID,NOMBRE,CATEGORIA,PRECIOPROVEEDOR,PRECIOVENTA,TEMPORADA
0,1,Playera Básica S,Playera,50,100,Verano
1,2,Playera Básica M,Playera,50,100,Verano
2,3,Playera Básica G,Playera,50,100,Verano
3,4,Playera Estampada S,Playera,60,120,Verano
4,5,Playera Estampada M,Playera,60,120,Verano


In [12]:
ordenes_productos_df.head(5)

,ORDENESPRODUCTOS_ID,ORDEN_ID,INVENTARIO_ID,CANTIDAD,PRECIOUNITARIO
0,1,1,1,2,100
1,2,1,2,1,100
2,3,2,4,2,120
3,4,2,5,1,120
4,5,3,7,2,140


In [13]:
ordenes_df.head(5)

,ORDEN_ID,CREADO_POR_ID,MODIFICADO_POR_ID,TIPOORDEN,ORGANIZACION,FECHACREACION,FECHAACEPTACION,FECHALIMITEPAGO,FECHAESTIMADAENTREGA,FECHAENTREGA,ENTREGAATIEMPO,ESTADO,TOTAL,METODOPAGO_ID,DESCUENTOAPLICADO,TIEMPOREPOSICION,TIEMPOENTREGA
0,1,1,1.0,Venta,RopaExpress,2025-05-28,2025-05-31,2025-06-07,2025-06-12,2025-06-13,True,Completado,1500,1,100,2,1
1,2,2,2.0,Venta,RopaExpress,2025-05-28,2025-05-30,2025-06-04,2025-06-07,2025-06-08,True,Pendiente,750,2,50,3,1
2,3,3,3.0,Venta,RopaExpress,2025-05-28,2025-06-03,2025-06-09,2025-06-14,2025-06-15,False,En Reparto,500,3,0,5,2
3,4,1,1.0,Venta,RopaExpress,2025-05-28,2025-06-01,2025-06-05,2025-06-09,2025-06-10,True,Completado,2000,1,200,1,1
4,5,2,2.0,Venta,RopaExpress,2025-05-28,2025-06-02,2025-06-06,2025-06-11,2025-06-12,False,Pendiente,1200,2,100,4,2


In [14]:
# Merge de DataFrames de HANA ML
# Realizar el merge de los DataFrames de HANA ML
# First, check column name format and case
inventario_id_col = [col for col in inventario_df.columns if col.upper() == 'INVENTARIO_ID'][0]
articulo_id_col = [col for col in articulo_df.columns if col.upper() == 'ARTICULO_ID'][0]
orden_id_col = [col for col in ordenes_df.columns if col.upper() == 'ORDEN_ID'][0]

# Now use the exact column names for merging
df = inventario_df.merge(historial_df, on=inventario_id_col, how='inner') \
                  .merge(articulo_df, on=articulo_id_col, how='inner') \
                  .merge(ordenes_productos_df, on=inventario_id_col, how='inner') \
                  .merge(ordenes_df, on=orden_id_col, how='inner')

# Mostrar las primeras filas del DataFrame resultante
print("\nPrimeras filas del DataFrame resultante:")
df.head(5)


Primeras filas del DataFrame resultante:


,INVENTARIO_ID,ARTICULO_ID,LOCATION_ID_x,STOCKACTUAL,IMPORTACION_x,EXPORTACION_x,STOCKMINIMO,STOCKRECOMENDADO,FECHAULTIMAIMPORTACION,FECHAULTIMAEXPORTACION,...,FECHALIMITEPAGO,FECHAESTIMADAENTREGA,FECHAENTREGA,ENTREGAATIEMPO,ESTADO,TOTAL,METODOPAGO_ID,DESCUENTOAPLICADO,TIEMPOREPOSICION_y,TIEMPOENTREGA
0,1,1,8,120,0,0,30,50,2025-04-01,2025-04-20,...,2025-06-07,2025-06-12,2025-06-13,True,Completado,1500,1,100,2,1
1,1,1,8,120,0,0,30,50,2025-04-01,2025-04-20,...,2025-06-04,2025-06-08,2025-06-09,False,En Reparto,650,2,30,5,2
2,2,2,8,100,0,0,25,50,2025-04-01,2025-04-19,...,2025-06-07,2025-06-12,2025-06-13,True,Completado,1500,1,100,2,1
3,2,2,8,100,0,0,25,50,2025-04-01,2025-04-19,...,2025-06-06,2025-06-10,2025-06-11,True,Pendiente,2100,1,0,3,1


In [16]:
# Since we already have pandas DataFrames, we'll work with them directly
# First, let's clean up the column names to handle any duplicates from the merge

# Check if we have the merged DataFrame from cell 13
if 'df' in globals() and len(df) > 0:
    # The df from cell 13 already contains the merged data
    # Let's select and rename the columns we need for analysis
    
    # Create a clean DataFrame with the columns we need
    df_clean = df[[
        'INVENTARIO_ID',
        'ARTICULO_ID', 
        'LOCATION_ID_x',  # Using _x version from inventario table
        'STOCKACTUAL',
        'STOCKMINIMO',
        'STOCKRECOMENDADO',
        'MARGENGANANCIA',
        'TIEMPOREPOSICION_x',
        'STOCKSEGURIDAD',
        'DEMANDAPROMEDIO',
        'ANIO',
        'MES', 
        'IMPORTACION_y',  # Using _y version from historial table
        'EXPORTACION_y',
        'STOCKSTART',
        'STOCKEND',
        'CATEGORIA',
        'PRECIOPROVEEDOR',
        'PRECIOVENTA',
        'TEMPORADA',
        'CANTIDAD',
        'PRECIOUNITARIO',
        'FECHACREACION',
        'FECHAENTREGA',
        'TIPOORDEN'
    ]].copy()
    
    # Rename columns to remove suffixes and have consistent naming
    df_clean = df_clean.rename(columns={
        'LOCATION_ID_x': 'LOCATION_ID',
        'IMPORTACION_y': 'IMPORTACION',
        'EXPORTACION_y': 'EXPORTACION',
        'TIEMPOREPOSICION_x': 'TIEMPOREPOSICION'
    })
    
    # Calculate delivery time in days
    df_clean['FECHACREACION'] = pd.to_datetime(df_clean['FECHACREACION'])
    df_clean['FECHAENTREGA'] = pd.to_datetime(df_clean['FECHAENTREGA'])
    df_clean['TIEMPOENTREGADIAS'] = (df_clean['FECHAENTREGA'] - df_clean['FECHACREACION']).dt.days
    
    # Replace the original df with the cleaned version
    df = df_clean
    
    print(f"Datos procesados correctamente. Dimensiones: {df.shape}")
    print("\nPrimeras 5 filas:")
    display(df.head())
    
    print(f"\nColumnas disponibles: {list(df.columns)}")
else:
    print("No hay datos disponibles para procesar")

Datos procesados correctamente. Dimensiones: (4, 26)

Primeras 5 filas:


,INVENTARIO_ID,ARTICULO_ID,LOCATION_ID,STOCKACTUAL,STOCKMINIMO,STOCKRECOMENDADO,MARGENGANANCIA,TIEMPOREPOSICION,STOCKSEGURIDAD,DEMANDAPROMEDIO,...,CATEGORIA,PRECIOPROVEEDOR,PRECIOVENTA,TEMPORADA,CANTIDAD,PRECIOUNITARIO,FECHACREACION,FECHAENTREGA,TIPOORDEN,TIEMPOENTREGADIAS
0,1,1,8,120,30,50,50,5,20,40,...,Playera,50,100,Verano,2,100,2025-05-28,2025-06-13,Venta,16
1,1,1,8,120,30,50,50,5,20,40,...,Playera,50,100,Verano,1,100,2025-05-28,2025-06-09,Venta,12
2,2,2,8,100,25,50,50,5,20,35,...,Playera,50,100,Verano,1,100,2025-05-28,2025-06-13,Venta,16
3,2,2,8,100,25,50,50,5,20,35,...,Playera,50,100,Verano,2,100,2025-05-28,2025-06-11,Venta,14



Columnas disponibles: ['INVENTARIO_ID', 'ARTICULO_ID', 'LOCATION_ID', 'STOCKACTUAL', 'STOCKMINIMO', 'STOCKRECOMENDADO', 'MARGENGANANCIA', 'TIEMPOREPOSICION', 'STOCKSEGURIDAD', 'DEMANDAPROMEDIO', 'ANIO', 'MES', 'IMPORTACION', 'EXPORTACION', 'STOCKSTART', 'STOCKEND', 'CATEGORIA', 'PRECIOPROVEEDOR', 'PRECIOVENTA', 'TEMPORADA', 'CANTIDAD', 'PRECIOUNITARIO', 'FECHACREACION', 'FECHAENTREGA', 'TIPOORDEN', 'TIEMPOENTREGADIAS']


In [18]:
# Preparar datos de ventas mensuales para modelado predictivo

# Primero preparamos el DataFrame existente para extracción de datos de ventas
# Asegurémonos que los datos de HANA ya estén en un DataFrame de pandas
if 'df' not in globals() or df is None:
    print("Error: No hay datos cargados desde HANA")
else:
    # Crear un DataFrame de ventas usando datos reales de la base de datos
    df_ventas = df.copy()
    
    # Agrupar por ARTICULO_ID, LOCATION_ID, ANIO y MES para calcular las ventas totales
    # Asumimos que EXPORTACION representa unidades vendidas
    df_ventas_agrupado = df_ventas.groupby(['ARTICULO_ID', 'LOCATION_ID', 'ANIO', 'MES'])[
        'EXPORTACION'].sum().reset_index()
    
    # Renombrar columnas para consistencia con el ejemplo
    df_ventas_agrupado = df_ventas_agrupado.rename(columns={
        'ARTICULO_ID': 'articulo_id',
        'LOCATION_ID': 'location_id',
        'ANIO': 'year',
        'MES': 'month',
        'EXPORTACION': 'unidades_vendidas'
    })
    
    # Crear columna de fecha para facilitar el análisis temporal
    df_ventas_agrupado['date'] = pd.to_datetime(
        df_ventas_agrupado['year'].astype(str) + '-' + 
        df_ventas_agrupado['month'].astype(str).str.zfill(2) + '-01'
    )
    
    print("=== Ventas agrupadas (primeras filas) ===")
    display(df_ventas_agrupado.head())
    print(f"Total de registros: {len(df_ventas_agrupado)}")

=== Ventas agrupadas (primeras filas) ===


,articulo_id,location_id,year,month,unidades_vendidas,date
0,1,8,2025,4,160,2025-04-01
1,2,8,2025,4,130,2025-04-01


Total de registros: 2


In [19]:
# Feature Engineering

# Podemos proceder con el feature engineering si tenemos datos
if 'df_ventas_agrupado' in globals() and len(df_ventas_agrupado) > 0:
    # Crear copia para el modelado
    df_model = df_ventas_agrupado.copy()
    
    # Transformaciones cíclicas para el mes (capturar estacionalidad)
    df_model['month_sin'] = np.sin(2 * np.pi * (df_model['month'] - 1) / 12)
    df_model['month_cos'] = np.cos(2 * np.pi * (df_model['month'] - 1) / 12)
    
    # Aplicar encoding a variables categóricas
    le_loc = LabelEncoder()
    le_prod = LabelEncoder()
    df_model['loc_enc'] = le_loc.fit_transform(df_model['location_id'])
    df_model['prod_enc'] = le_prod.fit_transform(df_model['articulo_id'])
    
    # Mostrar resultado del feature engineering
    print("=== Dataset con feature engineering ===")
    display(df_model.head())
    
    # Definir características (features) y variable objetivo
    features = ['loc_enc', 'prod_enc', 'year', 'month', 'month_sin', 'month_cos']
    target = 'unidades_vendidas'
    
    print(f"Features seleccionados: {features}")
    print(f"Variable objetivo: {target}")
else:
    print("No hay datos suficientes para realizar feature engineering")

=== Dataset con feature engineering ===


,articulo_id,location_id,year,month,unidades_vendidas,date,month_sin,month_cos,loc_enc,prod_enc
0,1,8,2025,4,160,2025-04-01,1.0,6.123234e-17,0,0
1,2,8,2025,4,130,2025-04-01,1.0,6.123234e-17,0,1


Features seleccionados: ['loc_enc', 'prod_enc', 'year', 'month', 'month_sin', 'month_cos']
Variable objetivo: unidades_vendidas


In [20]:
# Entrenamiento del modelo y predicciones

if 'df_model' in globals() and 'features' in globals() and len(df_model) > 0:
    # Identificar el último período de datos para separar entrenamiento y prueba
    ultimo_periodo = df_model['date'].max()
    print(f"Último período disponible: {ultimo_periodo}")
    
    # Separar datos de entrenamiento (todos menos el último mes)
    train_data = df_model[df_model['date'] < ultimo_periodo]
    test_data = df_model[df_model['date'] == ultimo_periodo].copy()
    
    print(f"Datos de entrenamiento: {len(train_data)} registros")
    print(f"Datos de prueba: {len(test_data)} registros")
    
    # Verificar que hay suficientes datos para entrenamiento
    if len(train_data) > 10:  # Umbral arbitrario, ajustar según necesidad
        # Preparar conjuntos de entrenamiento
        X_train = train_data[features]
        y_train = train_data[target]
        
        # Entrenar modelo XGBoost
        model = XGBRegressor(
            n_estimators=100,
            max_depth=3,
            learning_rate=0.1,
            random_state=42
        )
        model.fit(X_train, y_train)
        print("Modelo XGBoost entrenado correctamente")
        
        # Evaluar en datos de prueba si hay disponibles
        if len(test_data) > 0:
            X_test = test_data[features]
            y_test = test_data[target]
            y_pred = model.predict(X_test)
            
            # Convertir predicciones a enteros
            test_data['unidades_pred'] = y_pred.round().astype(int)
            
            # Mostrar métricas de precisión
            from sklearn.metrics import mean_absolute_error, mean_squared_error
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            
            print(f"Resultados de evaluación:")
            print(f"Error Absoluto Medio (MAE): {mae:.2f}")
            print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f}")
            
            # Mostrar predicciones vs valores reales
            comparison = test_data[['articulo_id', 'location_id', 'date', 'unidades_vendidas', 'unidades_pred']]
            display(comparison.head(10))
        else:
            print("No hay datos de prueba disponibles")
        
        # Preparar próximo periodo para predicción
        next_month_date = ultimo_periodo + pd.DateOffset(months=1)
        next_year = next_month_date.year
        next_month = next_month_date.month
        
        # Obtener lista única de articulos y locations
        unique_articles = df_model['articulo_id'].unique()
        unique_locations = df_model['location_id'].unique()
        
        print(f"\nPreparando predicciones para el próximo período: {next_month_date.strftime('%Y-%m-%d')}")
        print(f"Total de artículos: {len(unique_articles)}")
        print(f"Total de ubicaciones: {len(unique_locations)}")
        
        # Crear datos para predicción futura
        pred_list = []
        for loc in unique_locations:
            for prod in unique_articles:
                pred_list.append({
                    'location_id': loc,
                    'articulo_id': prod,
                    'date': next_month_date,
                    'year': next_year,
                    'month': next_month,
                    'month_sin': np.sin(2 * np.pi * (next_month - 1) / 12),
                    'month_cos': np.cos(2 * np.pi * (next_month - 1) / 12),
                    'loc_enc': le_loc.transform([loc])[0],
                    'prod_enc': le_prod.transform([prod])[0]
                })
        
        # Crear DataFrame para predicción
        df_next = pd.DataFrame(pred_list)
        X_next = df_next[features]
        
        # Hacer predicciones para el próximo período
        df_next['unidades_pred'] = model.predict(X_next).round().astype(int)
        
        print("\n=== Predicciones para el próximo período ===")
        display(df_next[['articulo_id', 'location_id', 'date', 'unidades_pred']].head(10))
        print(f"Total de predicciones: {len(df_next)}")
    else:
        print("No hay suficientes datos para entrenamiento")
else:
    print("No hay datos de modelo disponibles para entrenamiento")

Último período disponible: 2025-04-01 00:00:00
Datos de entrenamiento: 0 registros
Datos de prueba: 2 registros
No hay suficientes datos para entrenamiento


In [ ]:
# Análisis de inventario y generación de alertas

if 'df_next' in globals() and len(df_next) > 0:
    # Obtener datos de inventario actual
    # Nota: En este caso, ya tenemos df con esta información
    
    # Extraer las columnas relevantes de inventario
    df_inventory = df[['Articulo_ID', 'Location_ID', 'StockActual', 'StockMinimo']].copy()
    
    # Renombrar columnas para consistencia
    df_inventory = df_inventory.rename(columns={
        'Articulo_ID': 'articulo_id',
        'Location_ID': 'location_id',
        'StockActual': 'stock_actual',
        'StockMinimo': 'stock_min_actual'
    })
    
    # Eliminar duplicados si hay alguno (manteniendo un registro por artículo/ubicación)
    df_inventory = df_inventory.drop_duplicates(subset=['articulo_id', 'location_id'])
    
    print("=== Datos de inventario actual ===")
    display(df_inventory.head())
    
    # Unir datos de inventario con predicciones
    df_update = df_next.merge(df_inventory, on=['articulo_id', 'location_id'], how='left')
    
    # Calcular nuevo stock mínimo recomendado basado en predicciones
    # Añadimos un margen de seguridad del 20%
    df_update['stock_min_nuevo'] = (df_update['unidades_pred'] * 1.2).round().astype(int)
    
    # Generar alertas cuando el stock actual es menor que el stock mínimo nuevo
    df_update['alerta'] = df_update['stock_actual'] < df_update['stock_min_nuevo']
    
    # Mostrar resultados finales
    print("\n=== Análisis de inventario con alertas ===")
    display(df_update[['articulo_id', 'location_id', 'date', 
                      'stock_actual', 'stock_min_actual', 'unidades_pred',
                      'stock_min_nuevo', 'alerta']].head(10))
    
    # Mostrar solo productos con alerta
    productos_alerta = df_update[df_update['alerta'] == True]
    print(f"\n=== Productos que requieren atención ({len(productos_alerta)} productos) ===")
    if len(productos_alerta) > 0:
        display(productos_alerta[['articulo_id', 'location_id', 
                                 'stock_actual', 'stock_min_nuevo', 'unidades_pred']])
    else:
        print("No hay productos que requieran atención en este momento")
    
    # Calcular métricas generales
    total_articulos = len(df_update)
    porcentaje_alertas = (len(productos_alerta) / total_articulos * 100) if total_articulos > 0 else 0
    
    print(f"\nResumen:")
    print(f"- Total de productos analizados: {total_articulos}")
    print(f"- Productos que requieren reabastecimiento: {len(productos_alerta)} ({porcentaje_alertas:.1f}%)")
else:
    print("No hay datos de predicción disponibles para análisis de inventario")

In [ ]:
# Visualización de resultados y análisis gráfico

if 'df_update' in globals() and len(df_update) > 0:
    # 1. Gráfico de barras: Stock actual vs. Stock mínimo nuevo por ubicación
    plt.figure(figsize=(14, 8))
    
    # Agrupar por ubicación
    location_summary = df_update.groupby('location_id').agg({
        'stock_actual': 'sum',
        'stock_min_nuevo': 'sum',
        'alerta': 'sum'
    }).reset_index()
    
    location_summary['porcentaje_alertas'] = location_summary['alerta'] / df_update.groupby('location_id').size() * 100
    
    # Gráfico de barras agrupadas
    x = np.arange(len(location_summary))
    width = 0.35
    
    fig, ax1 = plt.subplots(figsize=(12, 6))
    
    # Barras para stock actual y stock mínimo
    bars1 = ax1.bar(x - width/2, location_summary['stock_actual'], width, label='Stock Actual')
    bars2 = ax1.bar(x + width/2, location_summary['stock_min_nuevo'], width, label='Stock Mínimo Nuevo')
    
    # Configurar eje Y1
    ax1.set_ylabel('Unidades')
    ax1.set_title('Análisis de Stock por Ubicación')
    ax1.set_xticks(x)
    ax1.set_xticklabels(location_summary['location_id'])
    ax1.legend(loc='upper left')
    
    # Añadir valores a las barras
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax1.annotate(f'{height}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom')
    
    add_labels(bars1)
    add_labels(bars2)
    
    # Segundo eje Y para porcentaje de alertas
    ax2 = ax1.twinx()
    ax2.plot(x, location_summary['porcentaje_alertas'], 'ro-', label='% Productos en Alerta')
    ax2.set_ylabel('Porcentaje de Productos en Alerta (%)')
    ax2.legend(loc='upper right')
    
    fig.tight_layout()
    plt.show()
    
    # 2. Histograma de la diferencia entre stock actual y predicción
    plt.figure(figsize=(12, 6))
    df_update['diferencia_stock'] = df_update['stock_actual'] - df_update['unidades_pred']
    
    # Determinar un umbral para colorear
    df_update['estado'] = 'Adecuado'
    df_update.loc[df_update['diferencia_stock'] < 0, 'estado'] = 'Déficit'
    df_update.loc[df_update['diferencia_stock'] > df_update['unidades_pred'], 'estado'] = 'Exceso'
    
    # Crear paleta de colores
    colors = {'Déficit': 'red', 'Adecuado': 'green', 'Exceso': 'orange'}
    
    # Agrupar por estado para histograma
    for estado, grupo in df_update.groupby('estado'):
        plt.hist(grupo['diferencia_stock'], alpha=0.6, label=estado, color=colors[estado], bins=15)
    
    plt.axvline(0, color='k', linestyle='--')
    plt.title('Distribución de la Diferencia entre Stock Actual y Ventas Predichas')
    plt.xlabel('Stock Actual - Ventas Predichas')
    plt.ylabel('Número de Productos')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # 3. Gráfico de dispersión: Stock actual vs Predicción, coloreado por alerta
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(df_update['unidades_pred'], 
                         df_update['stock_actual'],
                         c=df_update['alerta'].astype(int),
                         cmap='coolwarm', alpha=0.6, s=50)
    
    # Añadir línea diagonal de referencia (y=x)
    max_val = max(df_update['unidades_pred'].max(), df_update['stock_actual'].max()) * 1.1
    plt.plot([0, max_val], [0, max_val], 'k--', alpha=0.3)
    
    plt.colorbar(scatter, label='Alerta (1=Sí, 0=No)')
    plt.title('Stock Actual vs. Ventas Predichas')
    plt.xlabel('Ventas Predichas para el Próximo Período')
    plt.ylabel('Stock Actual')
    plt.grid(True, alpha=0.3)
    
    # Añadir etiquetas para algunos puntos destacados (top 5 alertas)
    highlight = df_update[df_update['alerta']].sort_values('stock_min_nuevo', ascending=False).head(5)
    for i, row in highlight.iterrows():
        plt.annotate(f"{row['articulo_id']}/{row['location_id']}",
                    (row['unidades_pred'], row['stock_actual']),
                    xytext=(5, 5), textcoords='offset points')
    
    plt.tight_layout()
    plt.show()
else:
    print("No hay datos suficientes para generar visualizaciones")

In [ ]:
# Recomendaciones y conclusiones automatizadas

if 'df_update' in globals() and len(df_update) > 0:
    # Generar insights automáticos basados en los resultados
    
    # 1. Productos críticos (mayor diferencia negativa entre stock actual y predicción)
    productos_criticos = df_update[df_update['alerta']].sort_values(
        by=['stock_actual'] - df_update[df_update['alerta']]['unidades_pred']
    ).head(5)
    
    # 2. Productos con exceso de stock
    productos_exceso = df_update[df_update['stock_actual'] > 2 * df_update['unidades_pred']].sort_values(
        by='stock_actual', ascending=False
    ).head(5)
    
    # 3. Cálculo de métricas generales de inventario
    valor_total_inventario = (df_update['stock_actual'] * df_update['unidades_pred']).sum()
    valor_productos_alerta = df_update[df_update['alerta']]['unidades_pred'].sum()
    productos_sin_rotacion = len(df_update[df_update['unidades_pred'] == 0])
    
    # 4. Análisis por ubicación
    ubicaciones_criticas = df_update.groupby('location_id')['alerta'].mean().sort_values(ascending=False)
    
    # Mostrar insights
    print("\n=== RECOMENDACIONES Y CONCLUSIONES AUTOMATIZADAS ===")
    
    print("\n1. PRODUCTOS QUE REQUIEREN ATENCIÓN INMEDIATA:")
    if len(productos_criticos) > 0:
        display(productos_criticos[['articulo_id', 'location_id', 'stock_actual', 
                                   'unidades_pred', 'stock_min_nuevo']])
        
        # Calcular cuánto inventario adicional se necesita
        productos_criticos['inventario_adicional'] = productos_criticos['stock_min_nuevo'] - productos_criticos['stock_actual']
        inventario_adicional_total = productos_criticos['inventario_adicional'].sum()
        
        print(f"\nSe necesita adquirir un total de {inventario_adicional_total} unidades adicionales")
        print("para los productos críticos identificados.")
    else:
        print("No se identificaron productos que requieran atención inmediata.")
    
    print("\n2. PRODUCTOS CON EXCESO DE INVENTARIO:")
    if len(productos_exceso) > 0:
        display(productos_exceso[['articulo_id', 'location_id', 'stock_actual', 
                                'unidades_pred']])
        
        # Calcular exceso
        productos_exceso['exceso_inventario'] = productos_exceso['stock_actual'] - productos_exceso['stock_min_nuevo']
        exceso_total = productos_exceso['exceso_inventario'].sum()
        
        print(f"\nSe identificó un exceso de {exceso_total} unidades en total para los productos")
        print("con mayor sobrestock. Considere redistribuir este inventario o aplicar estrategias")
        print("de venta para reducir el exceso.")
    else:
        print("No se identificaron productos con exceso significativo de inventario.")
    
    print("\n3. ANÁLISIS POR UBICACIÓN:")
    display(pd.DataFrame(ubicaciones_criticas).rename(columns={'alerta': 'Porcentaje_Alertas'}))
    ubicacion_mas_critica = ubicaciones_criticas.idxmax()
    print(f"\nLa ubicación '{ubicacion_mas_critica}' es la más crítica con {ubicaciones_criticas.max()*100:.1f}% de")
    print("productos en alerta. Se recomienda priorizar las acciones de inventario en esta ubicación.")
    
    # 5. Mostrar recomendaciones generales
    print("\n4. RECOMENDACIONES GENERALES:")
    print("- Revisar los niveles de stock mínimos actuales y ajustarlos basados en las predicciones.")
    print("- Priorizar el reabastecimiento de los productos críticos identificados.")
    print("- Considerar promociones para productos con exceso de inventario.")
    print("- Analizar la posibilidad de redistribuir inventario entre ubicaciones.")
    print("- Establecer un sistema de alertas automáticas basado en las predicciones mensuales.")
    
    # 6. Impacto financiero estimado
    print("\n5. IMPACTO FINANCIERO ESTIMADO:")
    print(f"- Valor total del inventario analizado: {valor_total_inventario} unidades")
    print(f"- Valor de productos en riesgo de stockout: {valor_productos_alerta} unidades ({valor_productos_alerta/valor_total_inventario*100:.1f}%)")
    print(f"- Productos sin rotación prevista: {productos_sin_rotacion} ({productos_sin_rotacion/len(df_update)*100:.1f}% del catálogo)")
else:
    print("No hay datos suficientes para generar recomendaciones")

In [ ]:
# Guardar el modelo y resultados para su uso posterior

import pickle
import os

if 'model' in globals() and 'df_update' in globals():
    # Crear directorio de modelos si no existe
    models_dir = '../models'
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)
    
    # Guardar el modelo entrenado
    model_path = os.path.join(models_dir, 'inventory_prediction_model.pkl')
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    
    # Guardar encoders para uso futuro
    encoders_path = os.path.join(models_dir, 'inventory_encoders.pkl')
    encoders = {
        'location_encoder': le_loc,
        'product_encoder': le_prod
    }
    with open(encoders_path, 'wb') as f:
        pickle.dump(encoders, f)
    
    # Guardar resultados como CSV
    results_dir = '../models/results'
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    
    # Guardar predicciones
    predictions_path = os.path.join(results_dir, f'predictions_{datetime.datetime.now().strftime("%Y%m%d")}.csv')
    df_update.to_csv(predictions_path, index=False)
    
    # Guardar alertas
    alertas_path = os.path.join(results_dir, f'alertas_{datetime.datetime.now().strftime("%Y%m%d")}.csv')
    if 'productos_alerta' in globals() and len(productos_alerta) > 0:
        productos_alerta.to_csv(alertas_path, index=False)
    
    print(f"Modelo guardado en: {model_path}")
    print(f"Encoders guardados en: {encoders_path}")
    print(f"Predicciones guardadas en: {predictions_path}")
    print(f"Alertas guardadas en: {alertas_path if 'productos_alerta' in globals() and len(productos_alerta) > 0 else 'No hay alertas para guardar'}")
else:
    print("No hay modelo o resultados para guardar")

In [ ]:
# Función para manejo de errores en la consulta

def verificar_resultados_consulta(df_result):
    """
    Verifica y reporta problemas en los resultados de la consulta
    """
    # Verificar que hay datos
    if df_result is None or len(df_result) == 0:
        print("⚠️ ADVERTENCIA: No se recuperaron datos de la consulta")
        return False
    
    # Verificar columnas críticas
    columnas_criticas = ['Inventario_ID', 'Articulo_ID', 'StockActual', 'StockMinimo', 
                        'Anio', 'Mes', 'Importacion', 'Exportacion']
    
    columnas_faltantes = [col for col in columnas_criticas if col not in df_result.columns]
    
    if columnas_faltantes:
        print(f"⚠️ ADVERTENCIA: Faltan columnas críticas en el resultado: {columnas_faltantes}")
        return False
    
    # Verificar si hay datos suficientes para el análisis
    min_registros = 10  # Número mínimo de registros para un análisis significativo
    if len(df_result) < min_registros:
        print(f"⚠️ ADVERTENCIA: Datos insuficientes para análisis. Se encontraron {len(df_result)} registros, pero se necesitan al menos {min_registros}.")
        return False
    
    # Verificar valores nulos en columnas críticas
    for col in columnas_criticas:
        if col in df_result.columns and df_result[col].isnull().sum() > 0:
            pct_nulos = df_result[col].isnull().mean() * 100
            print(f"⚠️ ADVERTENCIA: La columna '{col}' tiene {df_result[col].isnull().sum()} valores nulos ({pct_nulos:.1f}%).")
    
    print("✅ Verificación de resultados completada")
    return True

# Aplicar verificación a los datos ya cargados
if 'df' in globals() and df is not None:
    print("\nVerificando calidad de los datos cargados...")
    verificar_resultados_consulta(df)
else:
    print("\n⚠️ No hay datos cargados para verificar")

# Para usar en celdas posteriores
def ejecutar_si_hay_datos(df, mensaje_error="No hay datos suficientes para continuar"):
    """
    Decorador para ejecutar una función solo si hay datos suficientes
    """
    def decorador(func):
        def wrapper(*args, **kwargs):
            if df is None or len(df) == 0:
                print(f"⚠️ {mensaje_error}")
                return None
            return func(*args, **kwargs)
        return wrapper
    return decorador

In [ ]:
# Check the column names in each DataFrame to identify the correct case/format
print("Columns in inventario_df:", inventario_df.columns.tolist())
print("Columns in historial_df:", historial_df.columns.tolist())
print("Columns in articulo_df:", articulo_df.columns.tolist())
print("Columns in ordenes_productos_df:", ordenes_productos_df.columns.tolist())
print("Columns in ordenes_df:", ordenes_df.columns.tolist())

# Weekly Automated Stock Minimum Update Pipeline

This notebook has been extended with an automated weekly pipeline that:

1. **Retrains the model** every week using the most up-to-date data from the database
2. **Predicts demand** for the upcoming week for each product-location combination
3. **Updates the `stock_minimo` values** in the database with calculated values (with safety margin)

## Implementation Files

The automated pipeline has been implemented in the following files:

- `mlops/pipelines/weekly_stock_update.py`: The main pipeline script that connects to the database, trains the model, makes predictions, and updates the database
- `mlops/utils/schedule_stock_updates.py`: A scheduler script that runs the pipeline weekly (configured for Monday at 1:00 AM)

## How to Run

To manually run the pipeline once:
```
python mlops/pipelines/weekly_stock_update.py
```

To start the weekly scheduler:
```
python mlops/utils/schedule_stock_updates.py
```

## Output Files

The pipeline generates the following output files:

- Trained models are saved in `mlops/models/` as pickle files
- Prediction results are saved in `mlops/models/results/` as CSV files
- Logs are stored in `mlops/logs/` for monitoring

See the README.md file in the mlops directory for more details.

# Integración con la Aplicación Web

El modelo de predicción de stock mínimo ha sido integrado con la aplicación web para permitir su ejecución programada y manual desde la interfaz de usuario:

## Componentes de la Integración

### Backend

1. **Programador de Tareas**
   - Implementado en `backend/services/stockUpdateScheduler.js`
   - Utiliza node-cron para programar la ejecución semanal del modelo
   - Lee la configuración desde el archivo Python de configuración

2. **API Endpoints**
   - Rutas definidas en `backend/routes/ml.js`
   - Controladores en `backend/controllers/mlModelController.js`
   - Funcionalidades:
     - Ejecución manual del pipeline
     - Visualización de logs
     - Consulta de próxima ejecución programada

3. **Inicialización en Servidor**
   - El programador se inicia automáticamente al arrancar el servidor
   - Configurado en `backend/server.js`

### Frontend

Se ha desarrollado una interfaz de usuario para administradores en:
`frontend/src/pages/admin/ml/ModelManagement.jsx`

Esta interfaz permite:
- Ver cuándo será la próxima actualización automática
- Ejecutar manualmente la actualización del modelo
- Visualizar los logs de ejecución

## Diagrama de Flujo

```
[Aplicación Web] ---- Programación ---> [Node Cron] ---- Ejecución ---> [Python Script] ---- Actualización ---> [Base de Datos]
       |
       +----- API Manual -----> [Python Script] ---- Actualización ---> [Base de Datos]
```

## Beneficios de la Integración

1. **Automatización completa**: El modelo se ejecuta sin intervención humana
2. **Supervisión centralizada**: Los administradores pueden monitorear desde la aplicación
3. **Ejecución manual**: Permite actualizaciones bajo demanda cuando sea necesario
4. **Transparencia**: Los logs son accesibles desde la interfaz web